In [11]:
import os
import sys
from tqdm import tqdm
sys.path.append('../../src/meta_rule/')
sys.path.append('../../')

In [12]:
# Simple exclusion test to show how the LNN does self-supervised learning on a conjunction neuron

from lnn import LNN, Formula, And
import torch

model = LNN(alpha=0.5, propositional=True, learning=True)
n, f = 10, 2 # (number of True inputs, rand number of False inputs)
f = torch.randint(1, n, (f,))

props = torch.stack([Formula.true()]*n)
props[f] = Formula.false()
op = Formula.true()
op_target = Formula.true()

for i in range(len(props)):
    model[str(i)][()] = props[i]
prop_str = list(map(str, list(range(len(props)))))
model['And'] = And(*prop_str, lnn=model)
model['And'][()] = op

print_graph = lambda model: model.downward(func='print_graph', weights=True) # print the entire graph + neuron weights

# coefficients flags [unsupervised (contradictions), supervised (targets)]
model.train(coefficients=torch.tensor([1., 0.])) 

# uncomment below to see graph updates at print_epoch, as the network trains
# model.train(print_epoch=1, verbose=True, prediction_fn=print_graph, coefficients=torch.tensor([1., 0.])) 

print_graph(model)

success = True
if str(model['And'].state()) != 'True':
    for i in range(len(props)):
        if model[str(i)].state() != str(bool((model['And'].weights[i]).round())):
            success = False
            print(i, model[str(i)].state(), '!=', str(bool((model['And'].weights[i]).round())))
if success: print('\nsuccess')


converged at epoch 3
loss: 2.0 
loss_terms: tensor([2., 0.]) 
inference converged: True 
weights converged: True 
bounds tightened: tensor([0.0000, 0.0116])
And
() True tensor([1.0000, 0.9097], grad_fn=<StackBackward>) 
weights: tensor([1., 1., 1., 1., 1., 1., 1., 0., 0., 1.], requires_grad=True)
9
() True tensor([1.0000, 0.9091], grad_fn=<StackBackward>) 
8
() Contradiction tensor([1., 0.], grad_fn=<StackBackward>) 
7
() Contradiction tensor([1., 0.], grad_fn=<StackBackward>) 
6
() True tensor([1.0000, 0.9091], grad_fn=<StackBackward>) 
5
() True tensor([1.0000, 0.9091], grad_fn=<StackBackward>) 
4
() True tensor([1.0000, 0.9091], grad_fn=<StackBackward>) 
3
() True tensor([1.0000, 0.9091], grad_fn=<StackBackward>) 
2
() True tensor([1.0000, 0.9091], grad_fn=<StackBackward>) 
1
() True tensor([1.0000, 0.9091], grad_fn=<StackBackward>) 
0
() True tensor([1.0000, 0.9091], grad_fn=<StackBackward>) 

success
